In [27]:
import pandas as pd
from neo4j import GraphDatabase
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
import ast


In [10]:
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "vampire-float-olivia-maestro-sleep-4222"

In [65]:
df = pd.read_csv("final_movie.csv")

df.head(5)

df = df[(df['genres'].str.strip() != "[]") & (df['Director'].str.strip() != "[]")]


In [67]:
# Sampling data
df_movies = df.copy()
sub_df = df_movies.sample(n=1000, random_state=42)


In [37]:
# get unique genres
genres = []
sub_df['genres'].apply(lambda row: genres.extend(ast.literal_eval(row)))


genres = set(genres)

In [38]:
# get unique director
directors = []
sub_df['Director'].apply(lambda row: directors.extend(ast.literal_eval(row)))


directors = set(directors)

In [40]:
directors

{' Yeon Sang-Ho',
 'A. Dean Bell',
 'A.J. Edwards',
 'Aaron Leong',
 'Aaron Lipstadt',
 'Aaron Rose',
 'Aaron Seltzer',
 'Abel Ferrara',
 'Adoor Gopalakrishnan',
 'Ahmed Imamović',
 'Aigars Grauba',
 'Aivars Freimanis',
 'Alain Corneau',
 'Alain Robbe-Grillet',
 'Alan Parker',
 'Alastair Siddons',
 'Albert Maysles',
 'Albert Pyun',
 'Albert Uderzo',
 'Alberto Rodríguez',
 'Aldo Lado',
 'Alec Gillis',
 'Alec Tuckman',
 'Alejandro Fadel',
 'Aleksandr Stolper',
 'Aleksi Mäkelä',
 'Alessandro Genovesi',
 'Alessandro Piva',
 'Alessandro Rak',
 'Alex Chandon',
 'Alex Cox',
 'Alex Jones',
 'Alex Mebane',
 'Alex Mechanik',
 'Alex Merkin',
 'Alex Proyas',
 'Alex Stockman',
 'Alexander Kluge',
 'Alexander Korda',
 'Alexandre Arcady',
 'Alexis Alexiou',
 'Alfonso Brescia',
 'Alfonso Corona',
 'Alfonso Corona Blake',
 'Alfred Hitchcock',
 'Alfred J. Goulding',
 'Alison Murray',
 'Allan Arkush',
 'Amando de Ossorio',
 'Anat Baron',
 'Anatole Litvak',
 'Andrea Štaka',
 'Andreas Prodromou',
 'Andrei 

In [14]:
sub_df.columns

Index(['title', 'genres', 'Director', 'vote_average', 'vote_count',
       'overview'],
      dtype='object')

In [11]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [43]:
# kill switch
def delete_all_data(tx):
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session() as session:
    session.write_transaction(delete_all_data)

driver.close()

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_98724/3803800313.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(delete_all_data)


In [44]:
def create_movie_node(tx, row):
    tx.run("""
    CREATE (:movie {
            title: $title,
            overview: $overview,
            vote_average: $vote_average, 
            vote_count: $vote_count

    })
    """, title=row['title'], overview=row['overview'], vote_average=row['vote_average'],
                vote_count=row['vote_count'])
    

with driver.session() as session:
    for index, row in sub_df.iterrows():
        session.write_transaction(create_movie_node, row)

    

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_98724/556782445.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_movie_node, row)


In [ ]:
def create_director_node(tx, row):
    pass